In [ ]:
import cv2
import numpy as np

In [ ]:
sr = cv2.dnn_superres.DnnSuperResImpl_create()

# Load the downloaded model
sr.readModel("ESPCN_x4.pb")
sr.setModel("espcn", 4)  # Using ESPCN model with 4x upscaling

print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
input_video_path = "/content/input_video.mp4"
cap = cv2.VideoCapture(input_video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * 4)  # Upscaling by 4x
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * 4)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for saving video



In [ ]:
output_video_path = "high_res_video.mp4"

In [ ]:
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * 4)  # Upscaling by 4x
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * 4)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for saving video

In [ ]:
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply Super-Resolution
    high_res_frame = sr.upsample(frame)

    # Save the frame
    out.write(high_res_frame)

    # Display the frame (optional)
    #cv2.imshow("Super-Resolution Video", high_res_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
def apply_color_correction(frame):
    """Apply Histogram Equalization and White Balance"""
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)  # Convert to LAB color space
    l, a, b = cv2.split(lab)
    l = cv2.equalizeHist(l)  # Equalize histogram of L channel
    corrected = cv2.merge([l, a, b])
    return cv2.cvtColor(corrected, cv2.COLOR_LAB2BGR)  # Convert back to BGR

def reduce_noise(frame):
    """Apply Non-Local Means Denoising"""
    return cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)

def enhance_sharpness(frame):
    """Apply Unsharp Masking for Sharpness Enhancement"""
    blurred = cv2.GaussianBlur(frame, (0, 0), 3)
    return cv2.addWeighted(frame, 1.5, blurred, -0.5, 0)

def stabilize_frame(prev_frame, current_frame):
    """Optical Flow-based Frame Stabilization"""
    if prev_frame is None:
        return current_frame
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    h, w = flow.shape[:2]
    flow_map = np.column_stack((np.repeat(np.arange(h), w), np.tile(np.arange(w), h))) + flow.reshape(-1, 2)
    return cv2.remap(current_frame, flow_map[:, 1].reshape(h, w).astype(np.float32),
                     flow_map[:, 0].reshape(h, w).astype(np.float32), cv2.INTER_LINEAR)

# ------------------------ Process Video Frames ------------------------ #

In [ ]:
prev_frame = None

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply Super-Resolution
    high_res_frame = sr.upsample(frame)

    # Apply Video Enhancements
    high_res_frame = apply_color_correction(high_res_frame)
    high_res_frame = reduce_noise(high_res_frame)
    high_res_frame = enhance_sharpness(high_res_frame)
    high_res_frame = stabilize_frame(prev_frame, high_res_frame)  # Frame stabilization
    prev_frame = high_res_frame  # Update previous frame

    # Save the frame
    out.write(high_res_frame)

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()

print("Super-resolution video processing completed with enhancements!")

Super-resolution video processing completed with enhancements!
